# Import libraries

In [1]:
# Step 1: Import libraries
import requests
import pandas as pd
import datetime
import time
import concurrent.futures

In [2]:
!pip install ipython-autotime # This package will allow us to measure the execution time of our code
 
%load_ext autotime

time: 167 µs (started: 2021-04-14 08:33:24 +00:00)


In [3]:
import json

time: 1.98 ms (started: 2021-04-14 08:33:24 +00:00)


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 23.6 s (started: 2021-04-14 08:33:24 +00:00)


# Data collection
We will collect data from the following dates:
- 2019/01/01 to 2019/02/01
- 2020/01/01 to 2020/01/01

### Set up date_ranges
We will declare two variables containing the two date ranges we want, in minute frequency.

In [5]:
# Set up date range for 2019
dates2019 = pd.date_range(start='2019-01-01', end='2019-02-01', freq='T')

time: 9.71 ms (started: 2021-04-14 08:33:47 +00:00)


In [6]:
# Check the number of timestamps for January 2019
len(dates2019)

44641

time: 6.61 ms (started: 2021-04-14 08:33:47 +00:00)


In [7]:
# Step 8b: Set up date range for 2020
dates2020 = pd.date_range(start='2020-01-01', end='2020-02-01', freq='T')

time: 6.6 ms (started: 2021-04-14 08:33:47 +00:00)


In [8]:
# Check the number of timestamps for January 2019
len(dates2020)

44641

time: 9.51 ms (started: 2021-04-14 08:33:49 +00:00)


### Define retrieveCameraJSON
We will define a function named retrieveCameraJSON that takes one argument (date_time) and returns a JSON object from a url associated with the declared date_time.

In [ ]:
def retrieveCameraJSON(date_time):

  # Declare the base URL
  base_url = 'https://api.data.gov.sg/v1/transport/traffic-images?date_time='

  # Reformat the date_time
  formatted_date = date_time.strftime("%Y-%m-%dT%H%%3A%M%%3A%S")

  # Combine the base URL with the formatted date_time
  current_url = 'https://api.data.gov.sg/v1/transport/traffic-images?date_time={}'.format(formatted_date)

  # Send a GET request to the API
  get_request = requests.get(current_url)

  # Store the JSON data of the API response
  data = json.loads(get_request.text)

  # Return the stored JSON data
  return data

time: 4.62 ms (started: 2021-04-02 22:44:39 +00:00)


### Use concurrency to retrieve JSON data from 2019

In [ ]:
# Make concurrent API calls for 2019 data

# Create a list to store the .result of completed futures
future_results_2019 = []

with concurrent.futures.ThreadPoolExecutor(max_workers=150) as executor:
    future_to_scraper = [executor.submit(retrieveCameraJSON, date) for date in dates2019]
    for future in concurrent.futures.as_completed(future_to_scraper):
        future_results_2019.append(future.result())

time: 26min 30s (started: 2021-04-02 12:10:59 +00:00)


In [ ]:
# Step 11b: Get length of list of JSON results
len(future_results_2019)

44641

time: 7.09 ms (started: 2021-04-02 12:37:43 +00:00)


In [ ]:
# Save the JSON results to Google Drive
with open('/content/drive/MyDrive/Colab Notebooks/UpLevel SG Traffic/2019data.json', 'w') as f:
    json.dump(future_results_2019, f)

### Get January 2019 DataFrame from JSON results

In [ ]:
# Define to_df function

def to_df(future_results):
  stripped = []
  for i in future_results:
    try:
      stripped.append(i['items'][0]['cameras'])
    except:
      pass

  df = pd.DataFrame()

  for i in stripped:
    df = df.append(pd.json_normalize(i))

  df_filtered = df.loc[df['camera_id'] == '1709']

  df_filtered = df_filtered.drop_duplicates(subset = "image")

  return df_filtered

time: 6.16 ms (started: 2021-04-02 22:59:38 +00:00)


In [ ]:
# Open the 2019 JSON results
with open('/content/drive/MyDrive/Colab Notebooks/UpLevel SG Traffic/2019data.json') as f:
  loaded_json = json.load(f)

In [ ]:
# Load the first part of the JSON results to a dataframe
# The results will be loaded in batches to prevent Google Colab from crashing
future_results_2019_df = to_df(loaded_json[0:9000])

# View the dataframe
future_results_2019_df

,timestamp,image,camera_id,location.latitude,location.longitude,image_metadata.height,image_metadata.width,image_metadata.md5
3,2019-01-01T00:06:45+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,a39889e186a037b4594e3b864882f8a2
11,2019-01-01T00:03:45+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,7892491f53adea153a2345de7d5e6873
16,2019-01-01T00:44:46+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,3142635344ba2ddfad4e090f5984ed19
4,2019-01-01T02:11:10+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,67e24f49e8f21bbd95c778c026346aea
6,2019-01-01T01:52:30+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,a2e5aac5b15f414a3113b86072507b96
...,...,...,...,...,...,...,...,...
8,2019-01-07T06:41:29+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,2be0a02dfc2f433a1fecde01ba65bb33
16,2019-01-07T06:23:49+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,0aab9313c5cbc97c8944e7a0ee999006
70,2019-01-07T06:01:48+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,99d1d1f31bbb01e8a523a9786f973db7
16,2019-01-07T06:38:29+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,709223aea41a86818e733fb84e6f4370


time: 25min 58s (started: 2021-04-02 12:45:56 +00:00)


In [ ]:
# Save unsorted future_results_2019_df for backup
future_results_2019_df.to_csv('/content/drive/MyDrive/Colab Notebooks/UpLevel SG Traffic/future_results_2019_df_01.csv')

time: 74.8 ms (started: 2021-04-02 13:13:41 +00:00)


In [ ]:
# Save the next set of 2019 JSON results to a dataframe
future_results_2019_df_02 = to_df(loaded_json[9000:18000])

# View the dataframe
future_results_2019_df_02

,timestamp,image,camera_id,location.latitude,location.longitude,image_metadata.height,image_metadata.width,image_metadata.md5
6,2019-01-07T06:40:29+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,47f67767eb25a92bea8d575918701a5b
11,2019-01-07T06:19:28+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,0fed94c22ab1651063483508f7afe774
80,2019-01-07T06:04:48+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,d91f68944bc885dccd949b8a76ecbf6f
8,2019-01-07T06:12:48+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,07dd02f265f4027b7de19142fb1d8f15
6,2019-01-07T05:49:27+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,f34a86f2171815cbeeb2571044ab4504
...,...,...,...,...,...,...,...,...
5,2019-01-13T12:12:12+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,9bc948e0b0dff463423203c0dbed136f
7,2019-01-13T11:29:30+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,ba70b1d90ce1b144ef9a17f6a59a74aa
81,2019-01-13T12:22:13+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,9d4bc49c8cd960797f976cc6820870bb
6,2019-01-13T12:33:13+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,655f572e000ff83e2990257df058f4c7


time: 25min 26s (started: 2021-04-02 13:15:45 +00:00)


In [ ]:
# Save unsorted future_results_2019_df_02 for backup
future_results_2019_df_02.to_csv('/content/drive/MyDrive/Colab Notebooks/UpLevel SG Traffic/future_results_2019_df_02.csv')

time: 75.6 ms (started: 2021-04-02 13:42:29 +00:00)


In [ ]:
# Save the next set of 2019 JSON results to a dataframe
future_results_2019_df_03 = to_df(loaded_json[18000:27000])

# View the dataframe
future_results_2019_df_03

,timestamp,image,camera_id,location.latitude,location.longitude,image_metadata.height,image_metadata.width,image_metadata.md5
9,2019-01-13T11:38:11+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,25764df13458c277aad26a10cce13dcc
11,2019-01-13T12:20:33+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,b357ffaeeb1aee710bff87f20f732d32
11,2019-01-13T11:48:31+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,1ab160af74295282da7ead1f39792796
6,2019-01-13T12:06:32+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,4a82cb5b86dc48f5bb0363376da04112
5,2019-01-13T12:01:32+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,590ac6f86b8d2e25c9b9c645998a2091
...,...,...,...,...,...,...,...,...
5,2019-01-19T17:23:39+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,561f7830beaee88d72dadb488d476158
2,2019-01-19T18:37:42+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,54f67378d5d0b7e1a7586353fae05829
8,2019-01-19T17:29:19+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,ccf9f120aa008a91ba9064c7ce37f373
6,2019-01-19T17:51:20+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,280bc8487606f437a0572eb4f8acefdc


time: 24min 27s (started: 2021-04-02 13:43:26 +00:00)


In [ ]:
# Save unsorted future_results_2019_df_03 for backup
future_results_2019_df_03.to_csv('/content/drive/MyDrive/Colab Notebooks/UpLevel SG Traffic/future_results_2019_df_03.csv')

time: 64.8 ms (started: 2021-04-02 14:09:03 +00:00)


In [ ]:
# Save the next set of 2019 JSON results to a dataframe
future_results_2019_df_04 = to_df(loaded_json[27000:36000])

# View the dataframe
future_results_2019_df_04

,timestamp,image,camera_id,location.latitude,location.longitude,image_metadata.height,image_metadata.width,image_metadata.md5
5,2019-01-19T17:45:40+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,60a79674773ed4e6e65f2095e7ae6421
8,2019-01-19T18:05:41+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,e943950be6c4963be7d8ba337f968f6c
4,2019-01-19T17:32:39+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,9422adb6a37b936de15a8c30b6c73e5d
13,2019-01-19T18:00:40+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,25996d146d412925c11c7e17845da639
7,2019-01-19T18:11:41+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,b56585765eb2a841a47829139c00dba8
...,...,...,...,...,...,...,...,...
11,2019-01-26T00:03:42+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,425028c400e5ccf0f9ca32412b518661
3,2019-01-26T00:30:44+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,340fee9d69578679a346399631f65e27
9,2019-01-26T00:26:43+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,77d2ed9f60db7661096fb3fe3aa3abe7
6,2019-01-25T23:10:40+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,e507c21e2f3ed80d199438436554d7db


time: 25min 27s (started: 2021-04-02 14:10:17 +00:00)


In [ ]:
# Save unsorted future_results_2019_df_04 for backup
future_results_2019_df_04.to_csv('/content/drive/MyDrive/Colab Notebooks/UpLevel SG Traffic/future_results_2019_df_04.csv')

time: 66.5 ms (started: 2021-04-02 14:35:47 +00:00)


In [ ]:
# Save the last set of 2019 JSON results to a dataframe
future_results_2019_df_05 = to_df(future_results_2019_05)

# View the dataframe
future_results_2019_df_05

,timestamp,image,camera_id,location.latitude,location.longitude,image_metadata.height,image_metadata.width,image_metadata.md5
13,2019-01-26T01:22:06+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,96c7fb24d6f7c91263fce1ada4b40ee2
17,2019-01-26T01:27:46+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,4f0e0f1e0611577192d4498d350f1c22
15,2019-01-26T02:27:49+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,64d48b17bf93814342f39d1271ddfb31
9,2019-01-26T02:24:09+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,36fe424a0c458b0869c3aa25b55ee4c1
12,2019-01-26T02:29:09+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,d2344c7f05a963347daf0a1250645c78
...,...,...,...,...,...,...,...,...
11,2019-01-31T23:59:47+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,20b727575a2fd8218aaf857370137bfb
10,2019-01-31T23:58:27+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,0c172e9da57d5e8e241d6ca05e25347e
14,2019-01-31T23:38:46+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,915153972e3c6c049d73f31060b1b0ee
14,2019-01-31T23:08:45+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,667b0a64dd40ad4ce167ce508266499e


time: 22min 50s (started: 2021-04-02 23:10:29 +00:00)


In [ ]:
# Save unsorted future_results_2019_df_05 for backup
future_results_2019_df_05.to_csv('/content/drive/MyDrive/Colab Notebooks/UpLevel SG Traffic/future_results_2019_df_05.csv')

time: 61.9 ms (started: 2021-04-03 00:00:14 +00:00)


In [ ]:
# Load the dataframes from Google Drive

future_results_2019_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/UpLevel SG Traffic/future_results_2019_df_01.csv')

future_results_2019_df_02 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/UpLevel SG Traffic/future_results_2019_df_02.csv')

future_results_2019_df_03 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/UpLevel SG Traffic/future_results_2019_df_03.csv')

future_results_2019_df_04 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/UpLevel SG Traffic/future_results_2019_df_04.csv')

time: 1.52 s (started: 2021-04-03 00:04:11 +00:00)


In [ ]:
# Combine the dataframes for 2019
future_results_2019_full = future_results_2019_df.append([future_results_2019_df_02, 
                                                          future_results_2019_df_03, 
                                                          future_results_2019_df_04, 
                                                          future_results_2019_df_05])

time: 16.4 ms (started: 2021-04-03 00:04:25 +00:00)


In [ ]:
# View the full 2019 dataframe
future_results_2019_full

,Unnamed: 0,timestamp,image,camera_id,location.latitude,location.longitude,image_metadata.height,image_metadata.width,image_metadata.md5
0,3.0,2019-01-01T00:06:45+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,a39889e186a037b4594e3b864882f8a2
1,11.0,2019-01-01T00:03:45+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,7892491f53adea153a2345de7d5e6873
2,16.0,2019-01-01T00:44:46+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,3142635344ba2ddfad4e090f5984ed19
3,4.0,2019-01-01T02:11:10+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,67e24f49e8f21bbd95c778c026346aea
4,6.0,2019-01-01T01:52:30+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,a2e5aac5b15f414a3113b86072507b96
...,...,...,...,...,...,...,...,...,...
11,NaN,2019-01-31T23:59:47+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,20b727575a2fd8218aaf857370137bfb
10,NaN,2019-01-31T23:58:27+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,0c172e9da57d5e8e241d6ca05e25347e
14,NaN,2019-01-31T23:38:46+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,915153972e3c6c049d73f31060b1b0ee
14,NaN,2019-01-31T23:08:45+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,667b0a64dd40ad4ce167ce508266499e


time: 31.8 ms (started: 2021-04-03 00:04:33 +00:00)


In [ ]:
# Drop duplicates from the 2019 dataframe based on identical image urls
future_results_2019_full = future_results_2019_full.drop_duplicates(subset = "image")

# Verify whether the duplicates were dropped
future_results_2019_full

,Unnamed: 0,timestamp,image,camera_id,location.latitude,location.longitude,image_metadata.height,image_metadata.width,image_metadata.md5
0,3.0,2019-01-01T00:06:45+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,a39889e186a037b4594e3b864882f8a2
1,11.0,2019-01-01T00:03:45+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,7892491f53adea153a2345de7d5e6873
2,16.0,2019-01-01T00:44:46+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,3142635344ba2ddfad4e090f5984ed19
3,4.0,2019-01-01T02:11:10+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,67e24f49e8f21bbd95c778c026346aea
4,6.0,2019-01-01T01:52:30+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,a2e5aac5b15f414a3113b86072507b96
...,...,...,...,...,...,...,...,...,...
11,NaN,2019-01-31T23:59:47+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,20b727575a2fd8218aaf857370137bfb
10,NaN,2019-01-31T23:58:27+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,0c172e9da57d5e8e241d6ca05e25347e
14,NaN,2019-01-31T23:38:46+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,915153972e3c6c049d73f31060b1b0ee
14,NaN,2019-01-31T23:08:45+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,667b0a64dd40ad4ce167ce508266499e


time: 54.9 ms (started: 2021-04-03 00:05:26 +00:00)


In [ ]:
# Drop the Unnamed: 0 column (the values in this column were the old row indices from the dataframes that were created in batches)
future_results_2019_full = future_results_2019_full.drop(columns='Unnamed: 0')

# Check whether the column was dropped
future_results_2019_full

,timestamp,image,camera_id,location.latitude,location.longitude,image_metadata.height,image_metadata.width,image_metadata.md5
0,2019-01-01T00:06:45+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,a39889e186a037b4594e3b864882f8a2
1,2019-01-01T00:03:45+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,7892491f53adea153a2345de7d5e6873
2,2019-01-01T00:44:46+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,3142635344ba2ddfad4e090f5984ed19
3,2019-01-01T02:11:10+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,67e24f49e8f21bbd95c778c026346aea
4,2019-01-01T01:52:30+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,a2e5aac5b15f414a3113b86072507b96
...,...,...,...,...,...,...,...,...
11,2019-01-31T23:59:47+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,20b727575a2fd8218aaf857370137bfb
10,2019-01-31T23:58:27+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,0c172e9da57d5e8e241d6ca05e25347e
14,2019-01-31T23:38:46+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,915153972e3c6c049d73f31060b1b0ee
14,2019-01-31T23:08:45+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,667b0a64dd40ad4ce167ce508266499e


time: 30.6 ms (started: 2021-04-03 00:07:41 +00:00)


### Sort the DataFrame by timestamp

In [ ]:
# Convert timestamps to datetime dtype
future_results_2019_full['timestamp'] = pd.to_datetime(future_results_2019_full['timestamp'])

# Check whether the conversion was performed
future_results_2019_full

,timestamp,image,camera_id,location.latitude,location.longitude,image_metadata.height,image_metadata.width,image_metadata.md5
0,2019-01-01 00:06:45+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,a39889e186a037b4594e3b864882f8a2
1,2019-01-01 00:03:45+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,7892491f53adea153a2345de7d5e6873
2,2019-01-01 00:44:46+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,3142635344ba2ddfad4e090f5984ed19
3,2019-01-01 02:11:10+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,67e24f49e8f21bbd95c778c026346aea
4,2019-01-01 01:52:30+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,a2e5aac5b15f414a3113b86072507b96
...,...,...,...,...,...,...,...,...
11,2019-01-31 23:59:47+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,20b727575a2fd8218aaf857370137bfb
10,2019-01-31 23:58:27+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,0c172e9da57d5e8e241d6ca05e25347e
14,2019-01-31 23:38:46+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,915153972e3c6c049d73f31060b1b0ee
14,2019-01-31 23:08:45+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,667b0a64dd40ad4ce167ce508266499e


time: 214 ms (started: 2021-04-03 00:08:17 +00:00)


In [ ]:
# Sort the DataFrame by timestamp
future_results_2019_full = future_results_2019_full.sort_values(['timestamp'])

# Show the sorted dataframe
future_results_2019_full

,timestamp,image,camera_id,location.latitude,location.longitude,image_metadata.height,image_metadata.width,image_metadata.md5
39,2018-12-31 23:59:44+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,48f34eaa179e51989c8f937631da1bce
125,2019-01-01 00:00:44+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,ed9a57e100a83156605b19ec84bdc2fe
65,2019-01-01 00:01:25+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,c2713943d69482eaa4191454c76af1c8
1,2019-01-01 00:03:45+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,7892491f53adea153a2345de7d5e6873
80,2019-01-01 00:05:25+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,fa2696a66b3c5bb982e26b765f2d1b98
...,...,...,...,...,...,...,...,...
18,2019-01-31 23:54:47+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,d070313c2ca6bde6c44a074d136dcf05
9,2019-01-31 23:56:27+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,41786bfddd62f3138dd9d71b6210af28
3,2019-01-31 23:57:27+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,afd6213f51bda947cc27b98082c21f53
10,2019-01-31 23:58:27+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,0c172e9da57d5e8e241d6ca05e25347e


time: 36.2 ms (started: 2021-04-03 00:08:31 +00:00)


### Export the DataFrame to a CSV file

In [ ]:
# Save the CSV file in Google Drive
future_results_2019_full.to_csv('/content/drive/MyDrive/Colab Notebooks/UpLevel SG Traffic/future_results_2019_full.csv')

time: 607 ms (started: 2021-04-03 00:09:00 +00:00)


### Perform data collection for the date range in 2020

In [ ]:
# Make concurrent API calls for 2019 data

# Create a list to store the .result of completed futures
future_results_2020 = []

with concurrent.futures.ThreadPoolExecutor(max_workers=150) as executor:
    future_to_scraper = [executor.submit(retrieveCameraJSON, date) for date in dates2020]
    for future in concurrent.futures.as_completed(future_to_scraper):
        future_results_2020.append(future.result())

# Execution took 26min 37s

time: 26min 37s (started: 2021-04-03 00:12:06 +00:00)


In [ ]:
# Save the first batch of 2020 results to a dataframe
future_results_2020_df_01 = to_df(future_results_2020[0:9000])

# Show the dataframe
future_results_2020_df_01

,timestamp,image,camera_id,location.latitude,location.longitude,image_metadata.height,image_metadata.width,image_metadata.md5
15,2020-01-01T00:18:25+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,9185b7a928c2db3a1f758945f1ce2109
6,2020-01-01T00:45:46+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,09060f070b61d8f007a237c6b9c4dc9a
11,2020-01-01T01:26:48+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,bc74aededb1513bf71173dbbae03b8c2
13,2020-01-01T02:14:30+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,e9c3a77a9dff18faa8ebe12d7dabeae0
12,2020-01-01T02:27:31+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,c4833a02c62e6c5135504fa6d092019d
...,...,...,...,...,...,...,...,...
6,2020-01-07T05:36:29+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,30916a36058ea7b3f8b7f7a6307a36a1
6,2020-01-07T06:01:50+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,adaf64e6bd1ce323ef7ba94ff8ef718e
6,2020-01-07T05:06:27+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,c2556eb1042d0b811cfe6af0c55d5ac2
8,2020-01-07T06:22:31+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,fdd0d1e137ba27c0479df4e53327d63b


time: 24min 46s (started: 2021-04-03 00:39:55 +00:00)


In [ ]:
# Save unsorted future_results_2020_df_01 for backup
future_results_2020_df_01.to_csv('/content/drive/MyDrive/Colab Notebooks/UpLevel SG Traffic/future_results_2020_df_01.csv')

time: 64.9 ms (started: 2021-04-03 01:05:48 +00:00)


In [ ]:
# Save the next batch of 2020 results to a dataframe
future_results_2020_df_02 = to_df(future_results_2020[9000:18000])

# Show the dataframe
future_results_2020_df_02

,timestamp,image,camera_id,location.latitude,location.longitude,image_metadata.height,image_metadata.width,image_metadata.md5
6,2020-01-07T06:12:11+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,f6571daf33ea9ef6435d9d1dca7017f2
75,2020-01-07T05:55:50+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,1e7279eb2807cb02a8af5131e74784b5
12,2020-01-07T06:27:11+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,75928805bf434554e76f1147ba6ebcff
4,2020-01-07T05:19:28+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,f642a607d9a84554ce4e56a7788ef4af
4,2020-01-07T05:43:09+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,41a481cf813c4e048ae9a51dc0e1e468
...,...,...,...,...,...,...,...,...
76,2020-01-13T11:55:18+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,7e558cb5337621504216e6212b2c7100
79,2020-01-13T12:04:38+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,89d1985c797d63a47e169f97e52efe16
7,2020-01-13T11:48:38+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,af1baac3c57aac4af7c9882a2bdc213c
81,2020-01-13T12:13:19+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,f93449ab5f1bc14cda18eee73f88aef6


time: 25min 3s (started: 2021-04-03 01:06:13 +00:00)


In [ ]:
# Save unsorted future_results_2020_df_02 for backup
future_results_2020_df_02.to_csv('/content/drive/MyDrive/Colab Notebooks/UpLevel SG Traffic/future_results_2020_df_02.csv')

time: 62.7 ms (started: 2021-04-03 01:58:57 +00:00)


In [ ]:
# Save the next batch of 2020 results to a dataframe
future_results_2020_df_03 = to_df(future_results_2020[18000:27000])

# Show the dataframe
future_results_2020_df_03

,timestamp,image,camera_id,location.latitude,location.longitude,image_metadata.height,image_metadata.width,image_metadata.md5
10,2020-01-13T12:56:41+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,960e67434e93512171d8e0881d6da3bf
9,2020-01-13T12:04:38+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,89d1985c797d63a47e169f97e52efe16
5,2020-01-13T12:22:19+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,9c353af59bb65f320acae3e475bccbfc
12,2020-01-13T11:57:38+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,defc10dab05311d8be5c231e7087e12f
77,2020-01-13T11:49:18+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,2b9ceb2200b3dd09c81337ba0b8c4f86
...,...,...,...,...,...,...,...,...
17,2020-01-19T17:44:24+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,96d50ff641dfb67bb0c30c8d9b1d9b24
7,2020-01-19T18:02:25+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,f0b475fa17f385fc2c49e83116d1d842
8,2020-01-19T17:32:44+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,e036a0ca5ff368890f9f4cc9bca742d2
11,2020-01-19T17:37:24+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,fb6c92daa56df2f4310ada0d54c088d4


time: 25min (started: 2021-04-03 01:59:36 +00:00)


In [ ]:
# Save unsorted future_results_2020_df_03 for backup
future_results_2020_df_03.to_csv('/content/drive/MyDrive/Colab Notebooks/UpLevel SG Traffic/future_results_2020_df_03.csv')

time: 64.1 ms (started: 2021-04-03 02:25:02 +00:00)


In [ ]:
# Save the next batch of 2020 results to a dataframe
future_results_2020_df_04 = to_df(future_results_2020[27000:36000])

# Show the dataframe
future_results_2020_df_04

,timestamp,image,camera_id,location.latitude,location.longitude,image_metadata.height,image_metadata.width,image_metadata.md5
11,2020-01-19T18:11:46+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,67364a25b5a678cfa2c8099964deafbf
6,2020-01-19T17:45:44+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,643ad135c5365be338b40d134de1cefa
4,2020-01-19T17:36:44+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,1931d5ec7216ee1619634c18f836dccb
3,2020-01-19T17:30:44+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,b157c758eb71c8238244016aec1886e8
15,2020-01-19T17:14:43+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,5d893530626c59acf74d343867ab6c3f
...,...,...,...,...,...,...,...,...
2,2020-01-25T23:45:36+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,79faf46fdb0466a5977559598110b2d1
11,2020-01-25T23:41:16+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,d035fd082616da9fad431b4ef83e287b
2,2020-01-25T23:57:36+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,aeb35b79857bdf6797f9522a8a04b26f
6,2020-01-25T23:53:36+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,f1d2e2f5557534eeaf62b1a53f2fde02


time: 25min 4s (started: 2021-04-03 02:25:53 +00:00)


In [ ]:
# Save unsorted future_results_2020_df_04 for backup
future_results_2020_df_04.to_csv('/content/drive/MyDrive/Colab Notebooks/UpLevel SG Traffic/future_results_2020_df_04.csv')

time: 61.4 ms (started: 2021-04-03 02:51:37 +00:00)


In [ ]:
# Save the last batch of 2020 results to a dataframe
future_results_2020_df_05 = to_df(future_results_2020[36000:])

# Show the dataframe
future_results_2020_df_05

,timestamp,image,camera_id,location.latitude,location.longitude,image_metadata.height,image_metadata.width,image_metadata.md5
2,2020-01-26T00:06:37+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,4216c4b700bd6f7f62e3061e66f91c86
12,2020-01-25T23:45:36+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,79faf46fdb0466a5977559598110b2d1
7,2020-01-26T00:02:37+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,e5ffc81fa60465996117c1a87a12d55b
6,2020-01-26T00:15:37+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,2d85387aae05e9ad32eb8be37a9d7dd1
8,2020-01-26T00:18:37+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,ea528edbaea884723ba0e3fe3d9c41c0
...,...,...,...,...,...,...,...,...
12,2020-01-31T23:59:10+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,2379ffd6af1904bdb1b33132d0884d66
11,2020-01-31T23:29:28+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,5861420dae76e8587894adbcbd9a78ac
6,2020-01-31T23:32:29+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,b3994f0993160779b34f40bcd0c62ee7
11,2020-01-31T23:56:30+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,24c4ff81022e653f45cfb8bd0af3e276


time: 22min 57s (started: 2021-04-03 02:52:06 +00:00)


In [ ]:
# Save unsorted future_results_2020_df_05 for backup
future_results_2020_df_05.to_csv('/content/drive/MyDrive/Colab Notebooks/UpLevel SG Traffic/future_results_2020_df_05.csv')

time: 64.2 ms (started: 2021-04-03 03:15:52 +00:00)


In [ ]:
# Combine the 2020 results into one dataframe
future_results_2020_full = future_results_2020_df_01.append([future_results_2020_df_02, 
                                                             future_results_2020_df_03, 
                                                             future_results_2020_df_04, 
                                                             future_results_2020_df_05])

time: 21.2 ms (started: 2021-04-03 03:16:20 +00:00)


In [ ]:
# Show the full dataframe for 2020
future_results_2020_full

,timestamp,image,camera_id,location.latitude,location.longitude,image_metadata.height,image_metadata.width,image_metadata.md5
15,2020-01-01T00:18:25+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,9185b7a928c2db3a1f758945f1ce2109
6,2020-01-01T00:45:46+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,09060f070b61d8f007a237c6b9c4dc9a
11,2020-01-01T01:26:48+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,bc74aededb1513bf71173dbbae03b8c2
13,2020-01-01T02:14:30+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,e9c3a77a9dff18faa8ebe12d7dabeae0
12,2020-01-01T02:27:31+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,c4833a02c62e6c5135504fa6d092019d
...,...,...,...,...,...,...,...,...
12,2020-01-31T23:59:10+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,2379ffd6af1904bdb1b33132d0884d66
11,2020-01-31T23:29:28+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,5861420dae76e8587894adbcbd9a78ac
6,2020-01-31T23:32:29+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,b3994f0993160779b34f40bcd0c62ee7
11,2020-01-31T23:56:30+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,24c4ff81022e653f45cfb8bd0af3e276


time: 33.1 ms (started: 2021-04-03 03:16:24 +00:00)


In [ ]:
# Drop duplicates based on identical image urls
future_results_2020_full = future_results_2020_full.drop_duplicates(subset = "image")

# Check whether duplicates were dropped
future_results_2020_full

,timestamp,image,camera_id,location.latitude,location.longitude,image_metadata.height,image_metadata.width,image_metadata.md5
15,2020-01-01T00:18:25+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,9185b7a928c2db3a1f758945f1ce2109
6,2020-01-01T00:45:46+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,09060f070b61d8f007a237c6b9c4dc9a
11,2020-01-01T01:26:48+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,bc74aededb1513bf71173dbbae03b8c2
13,2020-01-01T02:14:30+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,e9c3a77a9dff18faa8ebe12d7dabeae0
12,2020-01-01T02:27:31+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,c4833a02c62e6c5135504fa6d092019d
...,...,...,...,...,...,...,...,...
12,2020-01-31T23:59:10+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,2379ffd6af1904bdb1b33132d0884d66
11,2020-01-31T23:29:28+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,5861420dae76e8587894adbcbd9a78ac
6,2020-01-31T23:32:29+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,b3994f0993160779b34f40bcd0c62ee7
11,2020-01-31T23:56:30+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,24c4ff81022e653f45cfb8bd0af3e276


time: 69.3 ms (started: 2021-04-03 03:16:34 +00:00)


In [ ]:
# Convert timestamps to datetime dtype
future_results_2020_full['timestamp'] = pd.to_datetime(future_results_2020_full['timestamp'])

# Check whether the conversion was performed
future_results_2020_full

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,timestamp,image,camera_id,location.latitude,location.longitude,image_metadata.height,image_metadata.width,image_metadata.md5
15,2020-01-01 00:18:25+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,9185b7a928c2db3a1f758945f1ce2109
6,2020-01-01 00:45:46+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,09060f070b61d8f007a237c6b9c4dc9a
11,2020-01-01 01:26:48+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,bc74aededb1513bf71173dbbae03b8c2
13,2020-01-01 02:14:30+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,e9c3a77a9dff18faa8ebe12d7dabeae0
12,2020-01-01 02:27:31+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,c4833a02c62e6c5135504fa6d092019d
...,...,...,...,...,...,...,...,...
12,2020-01-31 23:59:10+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,2379ffd6af1904bdb1b33132d0884d66
11,2020-01-31 23:29:28+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,5861420dae76e8587894adbcbd9a78ac
6,2020-01-31 23:32:29+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,b3994f0993160779b34f40bcd0c62ee7
11,2020-01-31 23:56:30+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,24c4ff81022e653f45cfb8bd0af3e276


time: 227 ms (started: 2021-04-03 03:20:26 +00:00)


In [ ]:
# Sort the DataFrame by timestamp
future_results_2020_full = future_results_2020_full.sort_values(['timestamp'])

# Show the sorted dataframe
future_results_2020_full

,timestamp,image,camera_id,location.latitude,location.longitude,image_metadata.height,image_metadata.width,image_metadata.md5
12,2019-12-31 23:59:24+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,1cc0aed7f981fccabfe22c43229a48d5
5,2020-01-01 00:00:44+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,9daa712d0a50f989a8a698b6fc7dd33b
6,2020-01-01 00:01:44+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,d623bfdab733b966ade6e390f18cd7fb
9,2020-01-01 00:02:44+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,0ddb5b38083ebcce87c8228b746a7347
6,2020-01-01 00:03:24+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,fc4923fc71ec9a99940007cea41ecf23
...,...,...,...,...,...,...,...,...
8,2020-01-31 23:53:30+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,bb50d5020f94a869583863593c0308d8
9,2020-01-31 23:54:50+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,f54b2765d796c244350a4db077e085cf
11,2020-01-31 23:56:30+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,24c4ff81022e653f45cfb8bd0af3e276
18,2020-01-31 23:57:30+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,d85d5d9db869044c3f64b54fa5fcb41d


time: 38.5 ms (started: 2021-04-03 03:21:32 +00:00)


In [ ]:
# Save the full 2020 dataframe to a CSV file
future_results_2020_full.to_csv('/content/drive/MyDrive/Colab Notebooks/UpLevel SG Traffic/future_results_2020_full.csv')

time: 631 ms (started: 2021-04-03 03:21:46 +00:00)
